In [176]:
import os
import glob
import csv

import IPython
import IPython.display
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import ModelCheckpoint
from keras.preprocessing.sequence import TimeseriesGenerator


mpl.rcParams['figure.figsize'] = (20, 8)
mpl.rcParams['axes.grid'] = False
plt.style.use('seaborn-darkgrid')


In [177]:
data_2 = pd.read_csv('../output/Book1.csv')
# checking our dataframe once again
data_2.head()


,Time,Subcar_8,Subcar_9,Subcar_10,Subcar_11,Subcar_12,Target
0,1625554196,895.897874,876.208309,872.715876,874.082376,867.163768,0
1,1625554197,879.275270,863.391568,854.110063,829.752373,857.818745,0
2,1625554197,845.568448,824.869687,827.835732,836.459802,841.675115,0
3,1625554197,900.568709,894.215299,869.888499,872.570914,856.703566,0
4,1625554197,899.608804,876.256241,874.002288,852.099173,867.941242,0


In [180]:
data_3 = data_2[['Subcar_9', 'Subcar_10', 'Subcar_11', 'Subcar_12']]
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(data_3)
data_scaled
print(data_scaled)

[[0.80649835 0.7984141  0.79367047 0.81583857]
 [0.78886082 0.77206162 0.72871976 0.80221808]
 [0.73584963 0.73484777 0.73854724 0.77868853]
 ...
 [0.6861812  0.65586072 0.64922525 0.62648012]
 [0.47384844 0.43558139 0.40733843 0.40423239]
 [0.26466063 0.23762409 0.24526347 0.31907693]]


In [184]:
from sklearn.preprocessing import LabelEncoder
target = data_2.iloc[:,-1]

label_encoder = LabelEncoder()
target = label_encoder.fit_transform(target)

In [185]:

x_train, x_test, y_train, y_test = train_test_split(
    data_scaled, target, test_size=0.10, random_state=120, shuffle=False)  # false for timeseries
print('X_train.shape: ', x_train.shape)
print('X_test.shape: ', x_test.shape)

X_train.shape:  (876, 4)
X_test.shape:  (219, 4)


In [186]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression()
# training the model
clf.fit(x_train, y_train)

LogisticRegression()

In [187]:
y_pred = clf.predict(x_test)


In [188]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: {:.2f}%".format(accuracy * 100))

Accuracy: 47.03%
